In [6]:
!pip install -q -q -q spacy
!pip install -q -q -q more-itertools

In [12]:
!pip install -q -q -q unidecode

In [3]:
!python -m spacy download fr_core_news_lg

C:\ProgramData\Anaconda3\python.exe: No module named -q


In [2]:
import numpy as np
import pandas as pd
import spacy
import itertools
from functools import reduce  
import unidecode
import re

nlp = spacy.load("fr_core_news_lg")

un peu de NLP

In [10]:
doc = nlp('Une tendance de fond est elle en train de s’installer ? «La question se pose. Ce surrisque qu’on observe est-il constant au cours du temps, auquel cas on va avoir une accumulation, ou bien est-ce que cet effet va s’estomper au fur et à mesure ?» s’interroge à haute voix l’épidémiologiste Mircea Sofonea.')
print([(token.text, token.lemma_, token.pos_) for token in doc])

[('Une', 'un', 'DET'), ('tendance', 'tendance', 'NOUN'), ('de', 'de', 'ADP'), ('fond', 'fond', 'NOUN'), ('est', 'être', 'AUX'), ('elle', 'lui', 'PRON'), ('en', 'en', 'ADP'), ('train', 'train', 'NOUN'), ('de', 'de', 'ADP'), ('s’', 's’', 'PRON'), ('installer', 'installer', 'VERB'), ('?', '?', 'PUNCT'), ('«', '«', 'PUNCT'), ('La', 'le', 'DET'), ('question', 'question', 'NOUN'), ('se', 'se', 'PRON'), ('pose', 'pose', 'VERB'), ('.', '.', 'PUNCT'), ('Ce', 'ce', 'DET'), ('surrisque', 'surrisqu', 'ADJ'), ('qu’', 'qu’', 'PRON'), ('on', 'on', 'PRON'), ('observe', 'observe', 'VERB'), ('est', 'être', 'AUX'), ('-il', 'il', 'PRON'), ('constant', 'constant', 'ADJ'), ('au', 'au', 'ADP'), ('cours', 'cours', 'NOUN'), ('du', 'de', 'ADP'), ('temps', 'temps', 'NOUN'), (',', ',', 'PUNCT'), ('auquel', 'auquel', 'ADJ'), ('cas', 'cas', 'NOUN'), ('on', 'on', 'PRON'), ('va', 'aller', 'VERB'), ('avoir', 'avoir', 'VERB'), ('une', 'un', 'DET'), ('accumulation', 'accumulation', 'NOUN'), (',', ',', 'PUNCT'), ('ou', '

#### Chargement du fichier

In [20]:
identification = pd.read_csv('data/IDENTIFICATION.csv',sep=';', dtype={'ACCEPT':str,'NOM_DOSSIER':str})

/tmp/ipykernel_274/3177199949.py:1: DtypeWarning: Columns (13,21,28,30,45,49,53,58,65,68,72,74,76,82,84,86,90,97,101,112,116,117,139,149,159,160,166,167,168,169,170,171) have mixed types. Specify dtype option on import or set low_memory=False.
  identification = pd.read_csv('data/IDENTIFICATION.csv',sep=';', dtype={'ACCEPT':str,'NOM_DOSSIER':str})


In [1]:
identification.head() 

#### Traitement du langage

In [23]:
mots_a_chercher = ['deceder', 'malade', 'maladie', 'siret', 'siren', 'pacage', 'mort', 'telephone', 'reprise', 'repreneur', 'retraite',
                   'cessation', 'cesser', 'arret', 'dcd', 'retraite', 'stopper', 'decede', 'deces', 'exister', 'existe', 'sante', 'retraiter', 'liquidation', 'liquider']

In [24]:
variables_commentaires = [comment for comment in  identification.columns if "COMMENT" in comment]
variables_commentaires

['ACCEPT_COMMENT', 'COMMENT_IDENT', 'PROD_COMMENT', 'AUTRE_COMMENT']

In [25]:
temp = []
for var in variables_commentaires:
    df =  identification[~identification[var].isna()][['NOM_DOSSIER', var]].copy()
    df[var] = df[var].str.lower()
    print(var)
    print('Nombre de commentaires :', df.shape[0])
    # nlp.pipe permet d'appliquer un réseau de neurone pour lemmatiser le texte
    docs = list(nlp.pipe(df[var].values))
    tokens = [[unidecode.unidecode(item.lemma_) for item in doc] for doc in docs] 
    df[var+'_parser'] = list(map(lambda x: np.intersect1d(x,mots_a_chercher), tokens))
    df[var+'_parser'] = df[var+'_parser'].apply(lambda x: ' '.join(x))
    
    # j'utilise des regex pour trouver les numéros de téléphones
    telephone = df[~df[var].isna()][var].str.findall(r"\b0[1-9](?:[\s.-]?[0-9]){8}\b")
    df.loc[telephone.apply(len)>0,var+'_telephone'] = telephone[telephone.apply(len)>0].apply(lambda x:' '.join([re.sub(r'[^0-9]', '', i) for i in x]))
    
    siren = df[~df[var].isna()][var].str.findall(r"\b(?:[\s.-]?[0-9]){9}\b")
    df.loc[siren.apply(len)>0,var+'_siren'] = siren[siren.apply(len)>0].apply(lambda x:' '.join([re.sub(r'[^0-9]', '', i) for i in x]))

    siret = df[~df[var].isna()][var].str.findall(r"\b(?:[\s.-]?[0-9]){14}\b")
    df.loc[siret.apply(len)>0,var+'_siret'] = siret[siret.apply(len)>0].apply(lambda x:' '.join([re.sub(r'[^0-9]', '', i) for i in x]))

    #pacage = df[~df[var].isna()][var].str.findall(r"\b066\d{6}\b")
    #df.loc[pacage.apply(len)>0,var+'_pacage'] = pacage[pacage.apply(len)>0].apply(lambda x:' '.join([re.sub(r'[^0-9]', '', i) for i in x]))
    
    temp.append(df.drop(var, axis=1))
    
df_commentaires = reduce(lambda left, right: pd.merge(left , right, on = ["NOM_DOSSIER"], how = "outer"), temp)

ACCEPT_COMMENT
Nombre de commentaires : 603
COMMENT_IDENT
Nombre de commentaires : 4788
PROD_COMMENT
Nombre de commentaires : 59
AUTRE_COMMENT
Nombre de commentaires : 489


In [300]:
df_commentaires.dropna(how='all').to_csv('IDENTIFICATION_parser.csv', index=False, sep=';')